In [1]:
#https://ipython.readthedocs.io/en/stable/config/extensions/autoreload.html?highlight=autoreload
%load_ext autoreload
%autoreload 2

import json
import time
from datetime import datetime
import pandas as pd
from Kafka import KafkaWriter
from OpenWeatherMap import OpenWeatherMap
from confluent_kafka import Consumer

In [17]:
openWeatherMap = OpenWeatherMap()
kafkaForecast = KafkaWriter('weather.forecast')
kafkaCleaned = KafkaWriter('weather.cleaned')

def format_date(dt: str) -> str:
    return datetime.fromtimestamp(float(dt)).strftime('%d.%m.%Y %H:%M')


# load 'locations.json' into a json-object and return it
def load_locations() -> json:
    with open('locations.json', mode='r') as file:
        return json.load(file)

def get_transformed_forecast(cities: json) -> {}:
    forecasts = []

    for key in cities:
        city = cities[key]
        forecast = openWeatherMap.get_forecast(city)

        value = {
            'city': forecast['city']['name'],
            'weather': []
        }

        preds = []
        for pred in forecast['list']:
            preds.append({
                'dt': pred['dt_txt'],
                'temp': pred['main']['temp']
            })
        value['weather'] = preds

        forecasts.append(value)
    cities = {
        'cities': forecasts
    }
    return cities

# for each location, query openWeatherMap for the 5-day forecast and 
# store the the returned values in Kafka
def collect_forecast_data() -> None:
    try:
        print("Starting collection ...")

        cities = load_locations()

        while True:
            forecasts = get_transformed_forecast(cities)
            dt = format_date(datetime.timestamp(datetime.now()))

            kafkaForecast.store(dt, forecasts)

            print(f"Weather data collected at {dt}!")

            time.sleep(60)

    except KeyboardInterrupt:
        print("... collection stopped!")

In [19]:
collect_forecast_data()

Starting collection ...
Weather data collected at 23.08.2022 21:22!
... collection stopped!


In [15]:
kafkaForecast.store('test', 'test2')

In [8]:
json.loads(get_transformed_forecast(load_locations()))

{'cities': [{'city': 'Bregenz',
   'weather': [{'dt': '2022-08-23 18:00:00', 'temp': 24.11},
    {'dt': '2022-08-23 21:00:00', 'temp': 21.1},
    {'dt': '2022-08-24 00:00:00', 'temp': 17.39},
    {'dt': '2022-08-24 03:00:00', 'temp': 13.12},
    {'dt': '2022-08-24 06:00:00', 'temp': 15.35},
    {'dt': '2022-08-24 09:00:00', 'temp': 21.49},
    {'dt': '2022-08-24 12:00:00', 'temp': 25.83},
    {'dt': '2022-08-24 15:00:00', 'temp': 26.66},
    {'dt': '2022-08-24 18:00:00', 'temp': 20.25},
    {'dt': '2022-08-24 21:00:00', 'temp': 16.7},
    {'dt': '2022-08-25 00:00:00', 'temp': 15.64},
    {'dt': '2022-08-25 03:00:00', 'temp': 14.96},
    {'dt': '2022-08-25 06:00:00', 'temp': 16.88},
    {'dt': '2022-08-25 09:00:00', 'temp': 22.46},
    {'dt': '2022-08-25 12:00:00', 'temp': 25.07},
    {'dt': '2022-08-25 15:00:00', 'temp': 25.66},
    {'dt': '2022-08-25 18:00:00', 'temp': 20.62},
    {'dt': '2022-08-25 21:00:00', 'temp': 17.75},
    {'dt': '2022-08-26 00:00:00', 'temp': 16.94},
    {'dt'